In [15]:
# To access datasets stored in personal google drive: grant permissions to access drive.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# In order to open one recipe bearing file from the drive:

file = open("/content/drive/My Drive/CookingWithBert/recipes/1000.mmf", 'rb')
a_recipe = file.read().decode('ISO-8859-1')

print(a_recipe[:500])
print(a_recipe[-500:])

MMMMM----- Recipe via Meal-Master (tm) v8.05
 
      Title: "BE MINE" LOLLIPOPS
 Categories: Candies, Valentine
      Yield: 8 Servings
 
           Text only
 
  Source: Better Homes and Gardens, Febuary 1998 Prep time: 20 minutes
  Cook: 6 to 8 minutes
  
  2 1/2 to 3 1/2-inch round or heart-shaped metal cookie cutters 8 oz.
  assorted red, pink, and/or clearhard candies 35 to 60 (2 to 3 oz.)
  assorted small decorative candies, such as red cinnamon candies,
  small nonpareils, c
ed until ready to use.
  
  Source: Sunset Magazine. November, 1991  "Vegetables, pilaf, wild
  rice".
  
  Posted by SANDY BETH - WINONAPL <SANDRA_B@selco.lib.mn.us> to the
  Fatfree Digest [Volume 15 Issue 15] Feb. 15, 1995.
  
  Individual recipes copyrighted by originator. FATFREE Recipe
  collections copyrighted by Michelle Dick 1995. Formatted by Sue Smith,
  SueSmith9@aol.com using MMCONV. Archived through kindness of Karen
  Mintzias, km@salata.com.
  
  1.80á
 
MMMMM
 



In [17]:
# In order to open all of the recipe bearing files in the drive:

import glob

filenames = glob.glob("/content/drive/My Drive/CookingWithBert/recipes" + "/*.mmf")

recipes = ""
for filename in filenames:
    file = open(filename, 'rb')
    recipes += file.read().decode('ISO-8859-1')

print(recipes[:500])
print(recipes[-500:])


MMMMM----- Recipe via Meal-Master (tm) v8.05
 
      Title: "BE MINE" LOLLIPOPS
 Categories: Candies, Valentine
      Yield: 8 Servings
 
           Text only
 
  Source: Better Homes and Gardens, Febuary 1998 Prep time: 20 minutes
  Cook: 6 to 8 minutes
  
  2 1/2 to 3 1/2-inch round or heart-shaped metal cookie cutters 8 oz.
  assorted red, pink, and/or clearhard candies 35 to 60 (2 to 3 oz.)
  assorted small decorative candies, such as red cinnamon candies,
  small nonpareils, c
 from the cabbage and discard. Quarter
  the cabbage; cut out the core. Using a mandoline or a thin sharp
  knife, slice the cabbage lengthwise into 1/4-inch wide shreds.
  In a large bowl, toss the cabbage with the red onion, parsley, and the
  dressing.  At this point the slaw can be refrigerated for up to 1 hour
  before serving.  Add the basil, if desired, at the last minute.
  Nutritional info per serving: 59 calories; 1.9 G protein; 2.8 G fat;
  7.4 G carbohydrate.
 
MMMMM
 



In [18]:
RECIPE_SEPERATOR = "[END]"

lines = recipes.splitlines()
for i, line in enumerate(lines):
  if line.startswith("MMMM"):
    lines[i] = RECIPE_SEPERATOR

recipes = "\n".join(lines)
print (recipes[:2000])


[END]
 
      Title: "BE MINE" LOLLIPOPS
 Categories: Candies, Valentine
      Yield: 8 Servings
 
           Text only
 
  Source: Better Homes and Gardens, Febuary 1998 Prep time: 20 minutes
  Cook: 6 to 8 minutes
  
  2 1/2 to 3 1/2-inch round or heart-shaped metal cookie cutters 8 oz.
  assorted red, pink, and/or clearhard candies 35 to 60 (2 to 3 oz.)
  assorted small decorative candies, such as red cinnamon candies,
  small nonpareils, colored candy hearts, spice drops, and gumdrops
  Edible rose petals or other flower petals (optional) Lollipop sticks
  
  Place unwrapped hard candies in a heavy plastic bag, then place bag
  on top of folded towel and crush candies into small chunks wiht meat
  mallet or small hammer.
  
  Make only three or four lollipops at one time.  Line a baking sheet
  with foil.  Place desired cookie cutters on foil, at least 2 inches
  apart. Divide crushed candies evenly among cutters, approximately 1
  1/2 to 2 tablespoons per lollipop.  Candy layer sh

In [0]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Embedding
from tensorflow.python.keras.callbacks import ModelCheckpoint
import numpy as np


def text_to_char_features(text, sequence_length, step_size):
  """
    Transforms the raw text into training and fitting data.

    x: Train data is a vectorized list of sequences pulled from the text.
    y: fit data is the character following the corresponding sequence in x
    indices_char: dict used to transform back from index to character
  """
  chars = sorted(list(set(text)))
  print('total chars:', len(chars))
  char_indices = dict((c, i) for i, c in enumerate(chars))
  indices_char = dict((i, c) for i, c in enumerate(chars))

  # cut the text in semi-redundant sequences of maxlen characters
  maxlen = sequence_length
  step = step_size
  sequences = []
  next_chars = []
  for i in range(0, len(text) - maxlen, step):
      sequences.append(text[i: i + maxlen])
      next_chars.append(text[i + maxlen])
  print('nb sequences:', len(sequences))

  print('Vectorization...')
  # x = np.zeros((len(sequences), maxlen))
  x = np.zeros((len(sequences), maxlen, len(chars)))
  # y = np.zeros(len(sequences))
  y = np.zeros((len(sequences), len(chars)))
  for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
  
  return x, y, indices_char

def create_char_rnn(sequence_length, vocab_size):
  model = Sequential()
  # model.add(LSTM(200))
  # model.add(Dense(200))
  # model.add(Dense(1))
  model.add(LSTM(128, input_shape = (sequence_length, vocab_size)))
  model.add(Dense(vocab_size, activation = 'softmax'))
  return model

x, y, idx_to_char = text_to_char_features(a_recipe, 40, 3)

vocab_size = len(idx_to_char)
sequence_length = 40

checkpoint = ModelCheckpoint("/content/drive/My Drive/CookingWithBert/models/char_rnn_weights.hdf5",
 save_weights_only=True, monitor='loss', verbose=1,
 save_best_only=True, mode='auto', save_freq=1)

model = create_char_rnn(sequence_length, vocab_size)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(x, y, epoch=2, callbacks=[checkpoint])


total chars: 109
nb sequences: 420894
Vectorization...


KeyboardInterrupt: ignored

In [0]:
# load model from saved weight file
# so that training can continue between sessions and testing can happen at will
char_rnn = create_char_rnn(sequence_length, vocab_size)
char_rnn.load_weights("/content/drive/My Drive/CookingWithBert/models/char_rnn_weights.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.evaluate(x, y)

13153/13153 [==============================] - 39s 3ms/step - loss: 1.5771 - accuracy: 0.5517


[1.5770748853683472, 0.5517327189445496]

In [0]:
import nltk
from nltk import word_tokenize
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Embedding
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

def text_to_word_features(text, sequence_length, step_size):
  text = word_tokenize(text.lower())
  words = sorted(list(set(text)))
  print('total words:', len(words))
  word_indices = dict((c, i) for i, c in enumerate(words))
  indices_word = dict((i, c) for i, c in enumerate(words))

  # cut the text in semi-redundant sequences of maxlen characters
  maxlen = sequence_length
  step = step_size
  sequences = []
  next_words = []
  for i in range(0, len(text) - maxlen, step):
      sequences.append(text[i: i + maxlen])
      # sequences.append([word_indices[word] for word in text[i:i+ maxlen]])
      next_words.append(text[i + maxlen])
      # next_words.append(word_indices[text[i + maxlen]])
  print('nb sequences:', len(sequences))

  return sequences, next_words, word_indices

def create_word_rnn(sequence_length, vocab_size):
  model = Sequential()
  # model.add(LSTM(200))
  # model.add(Dense(200))
  # model.add(Dense(1))
  model.add(LSTM(128, input_shape = (sequence_length, vocab_size)))
  model.add(Dense(vocab_size, activation = 'softmax'))
  return model

def generator(sequences, next_words, word_indices, batch_size):
  for i in range(len(sequences)):
    sequence = sequences[i]
    x = np.zeros((batch_size, len(sequence), len(word_indices)))
    y = np.zeros((batch_size, len(word_indices)))
    for j in range(batch_size):
      for t, word in enumerate(sequence):
        x[j, t, word_indices[word]] = 1
      y[j, word_indices[word]] = 1
    if i == len(sequences) - 1:
      i = 0
    yield x, y
      

sequences, next_words, word_indices = text_to_word_features(a_recipe, 10, 1)

word_checkpoint = ModelCheckpoint("/content/drive/My Drive/CookingWithBert/models/word_rnn_weights.hdf5",
 save_weights_only=True, monitor='loss', verbose=1,
 save_best_only=True, mode='auto', save_freq=1)

early_stopping = EarlyStopping(monitor = 'val_acc', patience = 20)

word_vocab_size = len(word_indices)
word_sequence_length = 10

word_model = create_word_rnn(word_sequence_length, word_vocab_size)
word_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
word_model.summary()
word_model.fit_generator(generator(sequences, next_words, word_indices, 64), steps_per_epoch = int(len(sequences)/64) + 1, epochs=2, callbacks=[word_checkpoint, early_stopping])


Streaming output truncated to the last 5000 lines.
Epoch 00001: loss improved from 6.72292 to 6.72130, saving model to /content/drive/My Drive/CookingWithBert/models/word_rnn_weights.hdf5
2641/3649 [====================>.........] - ETA: 4:41 - loss: 6.7213 - accuracy: 0.1011WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.199376). Check your callbacks.

Epoch 00001: loss improved from 6.72130 to 6.71961, saving model to /content/drive/My Drive/CookingWithBert/models/word_rnn_weights.hdf5
2642/3649 [====================>.........] - ETA: 4:41 - loss: 6.7196 - accuracy: 0.1011WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.309957). Check your callbacks.

Epoch 00001: loss did not improve from 6.71961
2643/3649 [====================>.........] - ETA: 4:41 - loss: 6.7213 - accuracy: 0.1010WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.309957). Check your callbacks.

Epoch 0